<a href="https://colab.research.google.com/github/PietroCaforio/research-biocv-proj/blob/dev/unimodal_ct_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 147 (delta 72), reused 94 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 3.36 MiB | 28.21 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

### Train ResNet model

In [7]:
def train(model,lr=0.0001,weight_decay = 0.001, reduce_lr_factor = 0.1):
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  criterion = nn.CrossEntropyLoss()
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = reduce_lr_factor)
  # Training loop
  num_epochs = 40
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0

      for batch in train_loader:
          frames = batch['frame'].float().to(device)
          labels = batch['label'].long().to(device)

          optimizer.zero_grad()
          outputs = model(frames)
          loss = criterion(outputs.logits, labels)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

      # Validation loop
      model.eval()
      val_loss = 0.0
      correct = 0
      total = 0
      # Initialize counters for each class (G1, G2, G3)
      correct_per_class = [0, 0, 0]  # For G1, G2, G3
      total_per_class = [0, 0, 0]  # For G1, G2, G3

      with torch.no_grad():
          for batch in val_loader:
              frames = batch['frame'].float().to(device)
              labels = batch['label'].long().to(device)

              outputs = model(frames)
              loss = criterion(outputs.logits, labels)

              val_loss += loss.item()
              _, predicted = torch.max(outputs.logits, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

              # Calculate accuracy per class
              for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                  correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                  total_per_class[i] += (labels == i).sum().item()
      scheduler.step(val_loss)
      # Compute total accuracy and per-class accuracy
      total_accuracy = 100 * correct / total
      class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
      print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
      print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")

In [8]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [9]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Resnet-50

In [10]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [12]:
train(model)

Epoch 1, Loss: 0.8740859725703932
Validation Loss: 30.64374042749405, Total Accuracy: 55.70%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 2, Loss: 0.5644496013856914
Validation Loss: 35.23538245111704, Total Accuracy: 55.70%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 3, Loss: 0.3669623801969502
Validation Loss: 36.82030949294567, Total Accuracy: 52.35%
Accuracy per class - G1: 0.00%, G2: 87.35%, G3: 8.33%
Epoch 4, Loss: 0.22790472423785355
Validation Loss: 41.023547288030386, Total Accuracy: 53.69%
Accuracy per class - G1: 0.00%, G2: 82.53%, G3: 17.42%
Epoch 5, Loss: 0.14511780711273625
Validation Loss: 54.97954525463283, Total Accuracy: 54.03%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 27.27%
Epoch 6, Loss: 0.10258303733210858
Validation Loss: 67.71056483443826, Total Accuracy: 67.11%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 56.82%
Epoch 7, Loss: 0.07783378397867288
Validation Loss: 84.68816133886575, Total Accuracy: 65.44%
Accuracy per cl

### Resnet-18

In [13]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [15]:
train(model, weight_decay = 0.0)

Epoch 1, Loss: 0.09098072832590606
Validation Loss: 0.9117697385605424, Total Accuracy: 69.80%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 62.88%
Epoch 2, Loss: 0.0015506593264122685
Validation Loss: 1.2278970685787498, Total Accuracy: 70.81%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 65.15%
Epoch 3, Loss: 0.0008267165318750202
Validation Loss: 1.3056258806609549, Total Accuracy: 70.81%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 65.15%
Epoch 4, Loss: 0.00046768913596504914
Validation Loss: 1.2888075279304758, Total Accuracy: 68.79%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 60.61%
Epoch 5, Loss: 0.0006613402835919509
Validation Loss: 1.2339078239514492, Total Accuracy: 68.46%
Accuracy per class - G1: 0.00%, G2: 75.90%, G3: 59.09%
Epoch 6, Loss: 0.00033124147380912705
Validation Loss: 1.2843187843158375, Total Accuracy: 69.13%
Accuracy per class - G1: 0.00%, G2: 75.90%, G3: 60.61%
Epoch 7, Loss: 0.0003079637472058506
Validation Loss: 1.4537022618984339, Total Accuracy

### Resnet-34

In [16]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [18]:
train(model)

Epoch 1, Loss: 0.08929527408047898
Validation Loss: 0.6666478825733065, Total Accuracy: 82.55%
Accuracy per class - G1: 0.00%, G2: 80.12%, G3: 85.61%
Epoch 2, Loss: 0.003958682103037885
Validation Loss: 0.897023736173287, Total Accuracy: 70.47%
Accuracy per class - G1: 0.00%, G2: 79.52%, G3: 59.09%
Epoch 3, Loss: 0.0011246306670204518
Validation Loss: 1.0397143974667415, Total Accuracy: 77.52%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 80.30%
Epoch 4, Loss: 0.0005641031267969831
Validation Loss: 1.0470969722839072, Total Accuracy: 75.17%
Accuracy per class - G1: 0.00%, G2: 77.11%, G3: 72.73%
Epoch 5, Loss: 0.0003467685387913163
Validation Loss: 1.0205472612869926, Total Accuracy: 73.83%
Accuracy per class - G1: 0.00%, G2: 76.51%, G3: 70.45%
Epoch 6, Loss: 0.007047608834436189
Validation Loss: 1.1152766624407378, Total Accuracy: 71.14%
Accuracy per class - G1: 0.00%, G2: 77.71%, G3: 62.88%
Epoch 7, Loss: 0.06559875823257567
Validation Loss: 0.9279684297740459, Total Accuracy: 67.45